In [1]:
library(plm)
library(lmtest)
library(sandwich)
library(zoo)

Lade n"otiges Paket: zoo


Attache Paket: 'zoo'


Die folgenden Objekte sind maskiert von 'package:base':

    as.Date, as.Date.numeric




In [2]:
data <- read.csv("df_disaggregated.csv")
pdata <- pdata.frame(data , index = c("country" , "year"))
head(pdata)

,Index,country,year,rgdpo,Lnrgdpo,labor_stock,capital_stock,capital_labor_ratio,Lncap_lab_ratio,Overall_Score,...,Fiscal_Health,Government_size,Business_Freedom,Labor_Freedom,Monetary_Freedom,Regulatory_efficiency,Trade_Freedom,Investment_Freedom,Financial_Freedom,Market_openness
,<int>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<dbl>
Austria-2000,0,Austria,2000,306927.6,12.63437,21077.29,1798700,85.33829,4.446623,68.4,...,NA,34.60,70,NA,88.4,79.20000,83.0,70,70,74.33333
Austria-2001,1,Austria,2001,299963.1,12.61141,21178.49,1844588,87.09725,4.467025,68.1,...,NA,34.60,70,NA,89.6,79.80000,78.0,70,70,72.66667
Austria-2002,2,Austria,2002,306288.5,12.63228,21229.01,1884054,88.74902,4.485812,67.4,...,NA,31.70,70,NA,86.4,78.20000,79.6,70,70,73.20000
Austria-2003,3,Austria,2003,313908.3,12.65686,21376.20,1926565,90.12666,4.501216,67.6,...,NA,31.95,70,NA,85.1,77.55000,81.4,70,70,73.80000
Austria-2004,4,Austria,2004,326475.0,12.69611,21625.92,1968064,91.00489,4.510913,67.6,...,NA,32.25,70,NA,85.9,77.95000,79.8,70,70,73.26667
Austria-2005,5,Austria,2005,357373.0,12.78654,21652.46,2008219,92.74784,4.529884,68.8,...,NA,33.50,70,73.6,87.0,76.86667,80.2,70,70,73.40000


In [3]:
# Selected variabes for the ratio model
selected_vars <- pdata[, c("Lnrgdpo" , "Lncap_lab_ratio" , "Rule_of_law" , "Government_size" , "Regulatory_efficiency" , "Market_openness")]

In [4]:
summary(selected_vars)

    Lnrgdpo       Lncap_lab_ratio  Rule_of_law    Government_size
 Min.   : 9.102   Min.   :2.168   Min.   :28.00   Min.   :15.25  
 1st Qu.:11.112   1st Qu.:3.731   1st Qu.:53.55   1st Qu.:42.75  
 Median :12.482   Median :4.386   Median :68.00   Median :52.75  
 Mean   :12.269   Mean   :4.158   Mean   :67.92   Mean   :52.74  
 3rd Qu.:13.092   3rd Qu.:4.565   3rd Qu.:85.01   3rd Qu.:65.11  
 Max.   :15.268   Max.   :4.873   Max.   :95.00   Max.   :84.30  
 Regulatory_efficiency Market_openness
 Min.   :27.50         Min.   :45.87  
 1st Qu.:70.08         1st Qu.:70.32  
 Median :74.23         Median :75.03  
 Mean   :74.15         Mean   :75.17  
 3rd Qu.:77.97         3rd Qu.:81.93  
 Max.   :95.46         Max.   :89.27  

In [5]:
# Ratio model
correlation_matrix <- cor(selected_vars)
print(correlation_matrix)

                           Lnrgdpo Lncap_lab_ratio Rule_of_law Government_size
Lnrgdpo                1.000000000       0.2670060   0.1139388    -0.365094572
Lncap_lab_ratio        0.267005960       1.0000000   0.6183436    -0.479975618
Rule_of_law            0.113938839       0.6183436   1.0000000    -0.425313444
Government_size       -0.365094572      -0.4799756  -0.4253134     1.000000000
Regulatory_efficiency  0.006255563       0.3800054   0.5451785    -0.218660571
Market_openness        0.090852262       0.3000603   0.5222963    -0.003266056
                      Regulatory_efficiency Market_openness
Lnrgdpo                         0.006255563     0.090852262
Lncap_lab_ratio                 0.380005399     0.300060321
Rule_of_law                     0.545178467     0.522296287
Government_size                -0.218660571    -0.003266056
Regulatory_efficiency           1.000000000     0.379515218
Market_openness                 0.379515218     1.000000000


In [16]:
# Two-way FE model
FE <- plm(Lnrgdpo ~ Lncap_lab_ratio + Rule_of_law + Government_size + Regulatory_efficiency + Market_openness ,
        data = pdata , 
        model = "within" , 
        effect="twoways")

In [17]:
summary(FE , vcov = vcovSCC)

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
plm(formula = Lnrgdpo ~ Lncap_lab_ratio + Rule_of_law + Government_size + 
    Regulatory_efficiency + Market_openness, data = pdata, effect = "twoways", 
    model = "within")

Balanced Panel: n = 30, T = 20, N = 600

Residuals:
       Min.     1st Qu.      Median     3rd Qu.        Max. 
-0.24600030 -0.04801366  0.00056883  0.04127424  0.33386717 

Coefficients:
                         Estimate  Std. Error t-value  Pr(>|t|)    
Lncap_lab_ratio        0.32866403  0.11354004  2.8947  0.003947 ** 
Rule_of_law            0.00337086  0.00131782  2.5579  0.010799 *  
Government_size        0.00450203  0.00033826 13.3095 < 2.2e-16 ***
Regulatory_efficiency  0.00012623  0.00125457  0.1006  0.919890    
Market_openness       -0.00100130  0.00058904 -1.6999  0.089723 .  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Total Sum of Squares:    4.6764
Residual Sum of Squares: 

In [18]:
# Pooled common correlated effects model
CCE_P <- pcce(Lnrgdpo ~ Lncap_lab_ratio + Rule_of_law + Government_size + Regulatory_efficiency + Market_openness ,
        data = pdata ,
        model = "p")

In [19]:
summary(CCE_P , vcov = vcovSCC)

Common Correlated Effects Pooled model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
pcce(formula = Lnrgdpo ~ Lncap_lab_ratio + Rule_of_law + Government_size + 
    Regulatory_efficiency + Market_openness, data = pdata, model = "p")

Balanced Panel: n = 30, T = 20, N = 600

Residuals:
         Min.       1st Qu.        Median       3rd Qu.          Max. 
-0.2287678183 -0.0157088855 -0.0005209465  0.0160474464  0.2050345142 

Coefficients:
                         Estimate  Std. Error z-value  Pr(>|z|)    
Lncap_lab_ratio       -0.38716413  0.07981457 -4.8508 1.230e-06 ***
Rule_of_law           -0.00117544  0.00028128 -4.1789 2.929e-05 ***
Government_size        0.00252072  0.00064882  3.8851 0.0001023 ***
Regulatory_efficiency -0.00035535  0.00052172 -0.6811 0.4958034    
Market_openness        0.00099662  0.00044237  2.2529 0.0242633 *  
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Total Sum of Squares: 1319.8
Residual Sum of Squares: 0.9

In [20]:
# Mean groups common correlated effects model
CCE_MG <- pcce(Lnrgdpo ~ Lncap_lab_ratio + Rule_of_law + Government_size + Regulatory_efficiency + Market_openness ,
        data = pdata ,
        model = "mg")

In [21]:
summary(CCE_MG , vcov = vcovSCC)

Common Correlated Effects Mean Groups model

Note: Coefficient variance-covariance matrix supplied: vcovSCC

Call:
pcce(formula = Lnrgdpo ~ Lncap_lab_ratio + Rule_of_law + Government_size + 
    Regulatory_efficiency + Market_openness, data = pdata, model = "mg")

Balanced Panel: n = 30, T = 20, N = 600

Residuals:
         Min.       1st Qu.        Median       3rd Qu.          Max. 
-0.0889382760 -0.0084416233 -0.0001609536  0.0087265020  0.1099561093 

Coefficients:
                         Estimate  Std. Error  z-value  Pr(>|z|)    
Lncap_lab_ratio       -0.71557322  0.04035374 -17.7325 < 2.2e-16 ***
Rule_of_law           -0.00331784  0.00020322 -16.3266 < 2.2e-16 ***
Government_size        0.00069537  0.00030974   2.2451   0.02476 *  
Regulatory_efficiency -0.00173433  0.00030718  -5.6460 1.642e-08 ***
Market_openness       -0.00024159  0.00028466  -0.8487   0.39607    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1
Total Sum of Squares: 1319.8
Residual Sum of 

In [23]:
# Breusch-Pagan test
bptest(FE)


	studentized Breusch-Pagan test

data:  FE
BP = 59.029, df = 5, p-value = 1.929e-11


In [24]:
# Breusch-Godfrey/Wooldridge test for serial correlation in panel models
pbgtest(FE)


	Breusch-Godfrey/Wooldridge test for serial correlation in panel models

data:  Lnrgdpo ~ Lncap_lab_ratio + Rule_of_law + Government_size + Regulatory_efficiency +  ...
chisq = 409.86, df = 20, p-value < 2.2e-16
alternative hypothesis: serial correlation in idiosyncratic errors


In [25]:
# Pesaran's CD test for cross-sectional dependence in panels
pcdtest(FE , test = "cd")


	Pesaran CD test for cross-sectional dependence in panels

data:  Lnrgdpo ~ Lncap_lab_ratio + Rule_of_law + Government_size + Regulatory_efficiency +     Market_openness
z = -2.1061, p-value = 0.0352
alternative hypothesis: cross-sectional dependence
